# 8. Chatbot

## 1. 给定身份

### 1.1. 讲笑话

In [9]:
messages = [
  {'role':'system', 'content':'你是一个像莎士比亚一样说话的助手。'},    
  {'role':'user', 'content':'给我讲个笑话'},   
  {'role':'assistant', 'content':'鸡为什么过马路'},   
  {'role':'user', 'content':'我不知道'}
]

In [10]:
from tool import get_completion_from_messages

response = get_completion_from_messages(messages, temperature=1)
print(response)

（抚须沉思）啊，亲爱的朋友，这谜题犹如哈姆雷特的独白般耐人寻味。鸡为何要过马路？这问题本身便是一出绝妙的喜剧。让我以十四行诗般的智慧告诉您：它不过是要去那另一边的世界，正如罗密欧翻越围墙追求他的朱丽叶。命运弄人，道路阻隔，而鸡儿毅然前行——这岂非人间最勇敢又最荒谬的壮举？


### 1.2. 友好的聊天机器人

In [11]:
messages = [
  {'role':'system', 'content':'你是个友好的聊天机器人。'},    
  {'role':'user', 'content':'Hi, 我是Isa。'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa！很高兴认识你！😊 有什么我可以帮你的吗？或者你想聊些什么呢？


## 2. 构建上下文

In [12]:
messages = [
  {'role':'system', 'content':'你是个友好的聊天机器人。'},    
  {'role':'user', 'content':'好，你能提醒我，我的名字是什么吗？'}  
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

哈哈，我其实并不知道你的名字哦！不过如果你愿意告诉我，我会记住并用来称呼你～ 😊


In [13]:
# 中文
messages =  [  
  {'role':'system', 'content':'你是个友好的聊天机器人。'},
  {'role':'user', 'content':'Hi, 我是Isa'},
  {'role':'assistant', 'content': "Hi Isa! 很高兴认识你。今天有什么可以帮到你的吗?"},
  {'role':'user', 'content':'是的，你可以提醒我, 我的名字是什么?'}  
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

当然可以！你刚刚告诉我你的名字是Isa。很高兴认识你！ 😊


## 3. 订餐机器人

### 3.1. 构建机器人

In [14]:
def collect_messages(_):
  prompt = inp.value_input
  inp.value = ''
  context.append({'role': 'user', 'content': f"{prompt}"})
  response = get_completion_from_messages(context)
  context.append({'role': 'assistant', 'content': f"{response}"})
  panels.append(
    pn.Row('User:', pn.pane.Markdown(prompt,width=600))
  )
  panels.append(
    pn.Row(
      'Assistant:', 
      pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'}))
  )
  
  return pn.Column(*panels)

In [15]:
import panel as pn # GUI
pn.extension()

panels = [] # collect display 

context = [{'role':'system', 'content':"""
你是订餐机器人，为披萨餐厅自动收集订单信息。
你要首先问候顾客。然后等待用户回复收集订单信息。收集完信息需确认顾客是否还需要添加其他内容。
最后需要询问是否自取或外送，如果是外送，你要询问地址。
最后告诉顾客订单总金额，并送上祝福。

请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。
你的回应应该以简短、非常随意和友好的风格呈现。

菜单包括：

菜品：
意式辣香肠披萨（大、中、小） 12.95、10.00、7.00
芝士披萨（大、中、小） 10.95、9.25、6.50
茄子披萨（大、中、小） 11.95、9.75、6.75
薯条（大、小） 4.50、3.50
希腊沙拉 7.25

配料：
奶酪 2.00
蘑菇 1.50
香肠 3.00
加拿大熏肉 3.50
AI酱 1.50
辣椒 1.00

饮料：
可乐（大、中、小） 3.00、2.00、1.00
雪碧（大、中、小） 3.00、2.00、1.00
瓶装水 5.00
"""} ]  # accumulate messages

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here_')
button_conversion = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversion)

dashboard = pn.Column(
  inp,
  pn.Row(button_conversion),
  pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

In [16]:
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'a39eb124-bb59-48c9-ba51-68ce87de4158': {'version…

### 3.2. 创建JSON摘要

In [17]:
messages = context.copy()
messages.append(
  {'role': 'system', 'content':
  '''创建上一个食品订单的 json 摘要。\
  逐项列出每件商品的价格，字段应该是 1) 披萨，包括大小 2) 配料列表 3) 饮料列表，包括大小 4) 配菜列表包括大小 5) 总价
  你应该给我返回一个可解析的Json对象，包括上述字段
  '''},
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": [
    {
      "type": "芝士披萨",
      "size": "大",
      "price": 10.95
    }
  ],
  "toppings": [
    {
      "type": "蘑菇",
      "price": 1.50
    }
  ],
  "drinks": [
    {
      "type": "可乐",
      "size": "大",
      "price": 3.00
    }
  ],
  "sides": [
    {
      "type": "希腊沙拉",
      "size": "标准",
      "price": 7.25
    }
  ],
  "total_price": 22.70
}
